In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [18]:
batch_size = 100
learning_rate = 0.0005
num_epoch = 1
hidden_size = 10

In [16]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [48]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Sequential(
                        nn.Conv2d(1,8,3,padding=1),   # batch x 8 x 28 x 28
                        nn.BatchNorm2d(8),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2), 
                        nn.Conv2d(8,16,3,padding=1),  # batch x 16 x 14 x 14
                        nn.BatchNorm2d(16),
                        nn.ReLU(),
                        nn.MaxPool2d(2,2),
                        nn.Conv2d(16,32,3,padding=1),  # batch x 32 x 7 x 7
                        nn.ReLU(),)
        
        self.fc2_1 = nn.Sequential(
                        nn.Linear(32*7*7, 800),
                        nn.Linear(800, hidden_size),)
        
        self.fc2_2 = nn.Sequential(
                        nn.Linear(32*7*7, 800),
                        nn.Linear(800, hidden_size),)
        
        
        self.fc3 = nn.Sequential(
                        nn.Linear(hidden_size,800),
                        nn.BatchNorm1d(800),
                        nn.ReLU(),
                        nn.Linear(800,1568),
                        nn.ReLU(),)
        
        self.fc4 = nn.Sequential(
                        nn.ConvTranspose2d(32,16,3,2,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(16),
                        nn.ConvTranspose2d(16,8,3,2,1,1),
                        nn.ReLU(),
                        nn.BatchNorm2d(8),
                        nn.ConvTranspose2d(8,1,3,1,1),
                        nn.BatchNorm2d(1),)
        
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def encoder(self, x):
        #x = x.view(batch_size,-1)
        out = self.fc1(x)
        out = out.view(batch_size,-1)
        out = self.relu(out)
        mu = self.fc2_1(out)
        log_var = self.fc2_2(out)
                
        return mu,log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        out = self.fc3(z)
        out = self.relu(out)
        out = out.view(batch_size,32,7,7)
        out = self.fc4(out)
        out = self.sigmoid(out)
        out = out.view(batch_size,28,28,1)
        
        return out 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 1, 28, 28))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE()

In [34]:
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

In [35]:
vae

VAE(
  (fc1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
  )
  (fc2_1): Sequential(
    (0): Linear(in_features=1568, out_features=800, bias=True)
    (1): Linear(in_features=800, out_features=10, bias=True)
  )
  (fc2_2): Sequential(
    (0): Linear(in_features=1568, out_features=800, bias=True)
    (1): Linear(in_features=800, out_features=10, bias=True)
  )
  (fc3): Sequential(
    (0): Linear(in_features=10, 

In [47]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 1, 28, 28), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [41]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [42]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [49]:
for epoch in range(1, 20):
    train(epoch)
    test()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Using a target size (torch.Size([100, 1, 28, 28])) that is different to the input size (torch.Size([100, 28, 28, 1])) is deprecated. Please ensure they have the same size.
  after removing the cwd from sys.path.


Train Epoch: 1 [0/60000 (0%)]	Loss: 645.901836
Train Epoch: 1 [10000/60000 (17%)]	Loss: 642.946094
Train Epoch: 1 [20000/60000 (33%)]	Loss: 643.513281
Train Epoch: 1 [30000/60000 (50%)]	Loss: 642.351523
Train Epoch: 1 [40000/60000 (67%)]	Loss: 641.682344
Train Epoch: 1 [50000/60000 (83%)]	Loss: 643.044805
====> Epoch: 1 Average loss: 642.9926
====> Test set loss: 643.2455
Train Epoch: 2 [0/60000 (0%)]	Loss: 644.035586
Train Epoch: 2 [10000/60000 (17%)]	Loss: 644.240312
Train Epoch: 2 [20000/60000 (33%)]	Loss: 644.231367
Train Epoch: 2 [30000/60000 (50%)]	Loss: 644.052383
Train Epoch: 2 [40000/60000 (67%)]	Loss: 642.243047
Train Epoch: 2 [50000/60000 (83%)]	Loss: 643.975508
====> Epoch: 2 Average loss: 642.9936
====> Test set loss: 641.9545
Train Epoch: 3 [0/60000 (0%)]	Loss: 643.741406
Train Epoch: 3 [10000/60000 (17%)]	Loss: 644.107813
Train Epoch: 3 [20000/60000 (33%)]	Loss: 646.890078
Train Epoch: 3 [30000/60000 (50%)]	Loss: 643.984453
Train Epoch: 3 [40000/60000 (67%)]	Loss: 642.41